Optional, Any And Defaults

In [77]:
from pydantic import BaseModel, ValidationError

class User(BaseModel):
    name: str
    age: int
    
try:
    user1 = User(name="Alice") # exception occurs because age is required
    print(user1.age)
except ValidationError as e:
    print(e)

1 validation error for User
age
  Field required [type=missing, input_value={'name': 'Alice'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


In [78]:
from pydantic import BaseModel, ValidationError
from typing import Optional

class User(BaseModel):
    name: str
    age: Optional[int] = None
    
try:
    user1 = User(name="Alice")
    print(user1.age)
except ValidationError as e:
    print(e)

None


In [79]:
from typing import Any

class Mnemonic(BaseModel):
    requiredInt: int
    optionalInt: int = 10
    optionalNullableInt: Optional[int] = 10
    optionalAnyType: Any = 10

In [80]:
m = Mnemonic(requiredInt=22, optionalNullableInt=None,
             optionalAnyType="This value could be literally anything including None")
m

Mnemonic(requiredInt=22, optionalInt=10, optionalNullableInt=None, optionalAnyType='This value could be literally anything including None')

UUIDs And Default Factories

In [81]:
import uuid

class User(BaseModel):
    id: uuid.UUID
    name: str

try:
    user = User(id=uuid.uuid4(), name="Allison")
    # user = User(id="asdf-qwer-1234-zxcv", name="Allison") # would return an exception because valid UUID is expected here
    print(user)
except ValidationError as e:
    print(e)

id=UUID('31d95cf5-021d-4783-bec8-e04f795507d8') name='Allison'


In [82]:
from pydantic import Field
class User(BaseModel):
    id: uuid.UUID = Field(default_factory=uuid.uuid4) 
    #id: uuid.UUID = Field(default_factory=lambda: uuid.uuid4()) # using lambda is also possible here, though a bit longer
    name: str

try:
    user = User(name="Allison")
    print(user)
    user = User(id = uuid.uuid4(), name="Bob") # we still can assign id explicitly
    print(user)
except ValidationError as e:
    print(e)

id=UUID('e0bad788-d998-44b4-ad53-4a8812007e0b') name='Allison'
id=UUID('04d06756-c785-4733-9a96-3624fa5cf4e9') name='Bob'


Immutable attributes

In [83]:
from pydantic import BaseModel, ValidationError

class User(BaseModel):
    name: str
    age: int

    class Config:
        frozen = True

try:
    user = User(name="Alice", age=22)
    user.name = "Billy"
    print(user.name)
except ValidationError as e:
    print(e)

1 validation error for User
name
  Instance is frozen [type=frozen_instance, input_value='Billy', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/frozen_instance


In [84]:
from pydantic import BaseModel, ValidationError, ConfigDict

class User(BaseModel):
    model_config: ConfigDict = {"frozen": True}
    name: str
    age: int

try:
    user = User(name="Alice", age=22)
    user.name = "Billy"
    print(user.name)
except ValidationError as e:
    print(e)

1 validation error for User
name
  Instance is frozen [type=frozen_instance, input_value='Billy', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/frozen_instance


In [85]:
from pydantic import BaseModel, ValidationError
import uuid

class User(BaseModel):
    id: int = Field(default_factory=uuid.uuid4, frozen=True)
    name: str
    age: int

try:
    user = User(name="Alice", age=22)
    user.id = uuid.uuid4()
    print(user)
except ValidationError as e:
    print(e)

1 validation error for User
id
  Field is frozen [type=frozen_field, input_value=UUID('f2b9f684-8aa0-48e0-8fec-3ae0e6122dac'), input_type=UUID]
    For further information visit https://errors.pydantic.dev/2.5/v/frozen_field


Additional properties

In [86]:
from pydantic import BaseModel, Field
from uuid import UUID, uuid4

class Product(BaseModel):
    id: UUID = Field(frozen=True, default_factory=uuid4)
    name: str

try:
    product = Product(name="Chair", price=202.4) # pydantic just ignores irrelevant property
    print(product)
except ValueError as e:
    print(e)

id=UUID('3a621c88-44e6-4db6-ab83-1aedcfb9b6de') name='Chair'


In [87]:
from pydantic import BaseModel, Field
from uuid import UUID, uuid4

class Product(BaseModel):
    # model_config: ConfigDict = {"extra": "ignore"} # default value
    model_config: ConfigDict = {"extra": "forbid"}
    id: UUID = Field(frozen=True, default_factory=uuid4)
    name: str

try:
    # pydantic raises an exception because extra attributes are explicitly forbidden by config
    product = Product(name="Chair", price=202.4) 
    print(product)
except ValidationError as e:
    print(e)

1 validation error for Product
price
  Extra inputs are not permitted [type=extra_forbidden, input_value=202.4, input_type=float]
    For further information visit https://errors.pydantic.dev/2.5/v/extra_forbidden


In [88]:
from pydantic import BaseModel, Field
from uuid import UUID, uuid4

class Product(BaseModel):
    # model_config: ConfigDict = {"extra": "ignore"} # default value
    model_config: ConfigDict = {"extra": "allow"}
    id: UUID = Field(frozen=True, default_factory=uuid4)
    name: str

try:
    # pydantic raises no exception because extra attributes are explicitly allowed by config
    product = Product(name="Chair", price=202.4, weight=33) 
    # additional attributes are printed
    print(product)
except ValidationError as e:
    print(e)

id=UUID('22ade4ba-3f71-4932-b6ad-d1a1a33aa547') name='Chair' price=202.4 weight=33


Enumerations

In [89]:
from enum import Enum

class Color(Enum):
    RED = "red"
    GREEN = "green"
    BLUE = "blue"

Color.GREEN

class Item(BaseModel):
    name: str
    color: Color

try:
    item = Item(name="chair", color="red")
    # pydantic would raise an exception because "pink" is not a part of the Color enum
    # item = Item(name="chair", color="pink")
    print(item)
except ValidationError as e:
    print(e)

name='chair' color=<Color.RED: 'red'>


For Better Performance: Literals

In [90]:
from typing import Literal, Type
from pydantic import BaseModel, ValidationError

TrafficLight = Literal["red", "green", "blue"]

class ItemWithLiteral(BaseModel):
    name: str
    color: TrafficLight

try:
    item = ItemWithLiteral(name="Chair", color="red")
    # would cause an exception because "grey" is not a part of literal
    # item = ItemWithLiteral(name="Table", color="grey")
    print(item)
except ValidationError as e:
    print(e)

name='Chair' color='red'


In [100]:
# Demonstrate that literals are by far quicker than enums
from pydantic import TypeAdapter
from timeit import timeit

literal = Literal["red", "green", "blue"]
class Color(Enum):
    RED = "red"
    GREEN = "green"
    BLUE = "blue"

lit_adapter = TypeAdapter(literal)
enum_adapter = TypeAdapter(Color)

print("Check literal:", timeit(lambda: lit_adapter.validate_python("red"), number=10000))
print("Check enum:   ", timeit(lambda: enum_adapter.validate_python("red"), number=10000))

Check literal: 0.003871107999657397
Check enum:    0.011906678999366704
